
## 数据分析=>机器学习=>深度学习  

## 机器学习步骤
   1. 提出问题
   2. 理解数据
   3. 数据清洗
   4. 构建模型
   5. 模型评估
   6. 方案实施

### 1. 提出问题

In [4]:
# 什么样的人在泰坦尼克号上更容易存活

### 2. 理解数据

In [1]:
# 导入模块
import pandas as pd 
import numpy as np

In [13]:
X_train = pd.read_csv('./train.csv')
X_test = pd.read_csv('./test.csv')
print('训练集数据数据：',X_train.shape,'预测集数据数量：',X_test.shape)
total_df = X_train.append(X_test,ignore_index=True)
print('总数据集',total_df.shape)

训练集数据数据： (891, 12) 测试集数据数量： (418, 11)
总数据集 (1309, 12)


D:\Anaconda3\lib\site-packages\pandas\core\frame.py:7134: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return concat(


In [3]:
X_train.info()
'''
PassengerId id 乘客id
Survived 是否生存
Pclass 客舱登记
Name 姓名
Sex  性别
age 年龄
SibSp 同代直系亲属数
Parch 不同代直系亲属数
Ticket 船票编号
Fare 船票价格
Cabin 客舱号
Embarked 登录港口

'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [15]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


#### 数据清洗


   1. 选择子集
   2. 列名重命名
   3. 缺失值数据处理
   4. 数据类型转换
   5. 数据排序
   6. 异常值处理


   * Age 年龄 缺失值由平均值补充
   * Fare 船票价格缺失值由平均值填充
   * Embarked 登录港口缺失值由众数填充
   * Survived 是否生存为需要得到的结果，暂不考虑
   * Cabin 填充为u 表示为unknow

In [23]:
#  Age 年龄 缺失值由平均值补充
total_df['Age'] = total_df['Age'].fillna(total_df['Age'].mean())
#  Fare 船票价格缺失值由平均值填充
total_df['Fare'] = total_df['Fare'].fillna(total_df['Fare'].mean())
# Embarked 登录港口缺失值由众数填充,有以下克制登录港口最多为 'S'
total_df['Embarked'] = total_df['Embarked'].fillna('S')
# Cabin 填充为u 表示为unknow
total_df['Cabin'] = total_df['Cabin'].fillna('U')

In [18]:
total_df['Embarked'] .value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [29]:
total_df['Pclass'] .value_counts()

3    709
1    323
2    277
Name: Pclass, dtype: int64

In [19]:
total_df['Cabin'] .value_counts()

C23 C25 C27        6
B57 B59 B63 B66    5
G6                 5
F2                 4
F4                 4
                  ..
C82                1
F E46              1
D34                1
C95                1
E38                1
Name: Cabin, Length: 186, dtype: int64

### 特征工程

   * 在数据处理过程中需要用到特征工程，实现特征的提取和特征的选择的过程

  * 在特征工程中，对于分类文本类型，需要使用转换器，将其转换独热类型

  * 可以使用特征缩放将不同衡量的转换到0-1之间

### 特征提取

   1. 数值类型 可直接使用
   2. 时间序列 转换程单独年、月、日
   3. 分类数据 使用数值类别One hot编码

### 特征缩放
   1. 目标值不需要缩放
   2. 同比缩放所有属性，有最小-最大缩放和标准化
   3. 最小-最大缩放，又叫归一化，将值重新缩放到0-1之间，将值减去最小值并处以最大值和最小值差，
   4. 标准化 减去平均值，所以标准化均值总是0，然后处以方差，结果的分布具备单位方差，不同于归一化，标准化不将值绑定到特定范围，受异常值影响小

In [39]:
# 把 Embarked 和 Pclass 这俩列单独提出来转换为one hot类型
Embarked_df =  total_df[['Embarked']]
Pclass_df = total_df[['Pclass']]

In [35]:
# 通过转换器 一次完成俩个转换 文本--整数类型--独热类型
from  sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
Embarked_df_onehot = encoder.fit_transform(Embarked_df)
Pclass_df_onehot = encoder.fit_transform(Pclass_df)
Pclass_df_onehot

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [38]:
# 将以上np数组转换为df表格，并添加列名
Embarked_df_df = pd.DataFrame(Embarked_df_onehot,columns=['Embarked_C','Embarked_Q','Embarked_S'])
Pclass_df_df = pd.DataFrame(Pclass_df_onehot,columns=['Pclass_1','Pclass_2','Pclass_3'])

In [40]:
total_df['Name'] .value_counts()

Connolly, Miss. Kate                    2
Kelly, Mr. James                        2
Chevre, Mr. Paul Romaine                1
Panula, Mr. Ernesti Arvid               1
Albimona, Mr. Nassef Cassem             1
                                       ..
Gustafsson, Mr. Johan Birger            1
Burns, Miss. Mary Delia                 1
Nakid, Mr. Sahid                        1
Ware, Mr. John James                    1
de Messemaeker, Mr. Guillaume Joseph    1
Name: Name, Length: 1307, dtype: int64

#### 分数数据特征提取：姓名，客舱号

In [41]:
# 定义一个函数，从名字当中获取头衔
def get_title(name):
    str1 = name.split(',')[1] # 按逗号分隔，取名字的后半部分 Miss. Kate 
    str2 = str1.split('.')[0] # 按点好分割，取称号的前半部分
    str3 = str2.strip()
    return str3

In [69]:
# 使用map函数 对列表当中的每一个数据使用get_title 函数
name_ser =  total_df['Name'].map(get_title)
name_df = pd.DataFrame(name_ser)

In [70]:
name_df['Name'].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Jonkheer          1
Sir               1
Don               1
Dona              1
Capt              1
Mme               1
the Countess      1
Lady              1
Name: Name, dtype: int64

In [71]:
title_mapdict = {
    "Capt":'Officer',
    "Col":'Officer',
    "Major":'Officer',
    "Jonkheer":'Royalty',
    "Don":'Royalty',
    "Sir":'Royalty',
    "Dr":'Officer',
    "Rev":'Officer',
    "the Countess ":'Royalty',
    "Dona":'Royalty',
    "Mme":'Mrs',
    "Mlle":'Miss',
    "Ms":'Mrs',
    "Mr":'Mr',
    "Mrs":'Mrs',
    "Miss":'Miss',
    "Master":"Master",
    "Lady":'Royalty',
    
}
len(title_mapdict)

18

In [72]:
name_df["Name"] = name_df["Name"].map(title_mapdict)
name_df['Name'].value_counts()
# 将姓名转换为以下六种类型
Name_df_onehot = pd.get_dummies(name_df['Name'])
Name_df_onehot

,Master,Miss,Mr,Mrs,Officer,Royalty
0,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,1,0,0,0,0
3,0,0,0,1,0,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
1304,0,0,1,0,0,0
1305,0,0,0,0,0,1
1306,0,0,1,0,0,0
1307,0,0,1,0,0,0


In [74]:
# 提取客舱号信息，
Cabin_ser = total_df['Cabin'].map(lambda x :x[0])
Cabin_One = pd.get_dummies(Cabin_df,prefix='Cabin')
Cabin_One.head()

,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1


In [79]:
 # 提取家庭类别数据
Family_df  = pd.DataFrame()
'''
家庭人数 = 同代直系亲属 + 不同代直系亲属数 +1 （加自己）
SibSp 同代直系亲属数
Parch 不同代直系亲属数
'''
Family_df["Family_size"] = total_df['SibSp']+total_df['Parch']+1


,Family_size
0,2
1,2
2,1
3,2
4,1
...,...
1304,1
1305,1
1306,1
1307,1


In [81]:
Family_df["Family_size"].value_counts()

1     790
2     235
3     159
4      43
6      25
5      22
7      16
11     11
8       8
Name: Family_size, dtype: int64

In [83]:
'''
定义家庭类别
小家庭 Family_small 家庭人数为1
中家庭 Family_mid 家庭人数在2-4之间
大家庭  Family_large 家庭人数大于等于5

'''
Family_df['Family_small'] = Family_df["Family_size"].map(lambda x :1 if x ==1 else 0)
Family_df['Family_mid'] = Family_df["Family_size"].map(lambda x :1 if 2<=x <=4 else 0)
Family_df['Family_large'] = Family_df["Family_size"].map(lambda x :1 if x >=51 else 0)
Family_df.head()

,Family_size,Family_small,Family_mid,Family_large
0,2,0,1,0
1,2,0,1,0
2,1,1,0,0
3,2,0,1,0
4,1,1,0,0


In [121]:
# 将sex 中的male 转换为1，female转换为0
a =   np.array(total_df['Sex'])
a[np.where(a=='male')] = 1
a[np.where(a=='female')] = 0
a

array([1, 0, 0, ..., 1, 1, 1], dtype=object)

In [147]:
df1  =pd.DataFrame(a,dtype='int')
df1['Sex'] = df1[0]

In [148]:
df1['Sex'] 

0       1
1       0
2       0
3       0
4       1
       ..
1304    1
1305    0
1306    1
1307    1
1308    1
Name: Sex, Length: 1309, dtype: int32

In [105]:
total_df_final = pd.DataFrame()
total_df_final['age']= total_df['Age']

In [107]:
total_df_final = total_df_final.join(Cabin_One)
total_df_final = total_df_final.join(Embarked_df_df)
total_df_final = total_df_final.join(total_df[['Fare']])
total_df_final = total_df_final.join(Name_df_onehot)
total_df_final = total_df_final.join(Family_df)
total_df_final = total_df_final.join(Pclass_df_df)
total_df_final = total_df_final.join(total_df[['Survived']])
total_df_final = total_df_final.join(df1[['Sex']])

In [218]:
total_df_final = total_df_final.join(total_df[['Survived']])

In [298]:
total_df_final = total_df_final.join(total_df[['PassengerId']])

In [153]:
total_df_final

,age,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,...,Royalty,Family_size,Family_small,Family_mid,Family_large,Pclass_1,Pclass_2,Pclass_3,Survived,Sex
0,22.000000,0,0,0,0,0,0,0,0,1,...,0,2,0,1,0,0,0,1,0.0,1
1,38.000000,0,0,1,0,0,0,0,0,0,...,0,2,0,1,0,1,0,0,1.0,0
2,26.000000,0,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,1,1.0,0
3,35.000000,0,0,1,0,0,0,0,0,0,...,0,2,0,1,0,1,0,0,1.0,0
4,35.000000,0,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,29.881138,0,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,1,NaN,1
1305,39.000000,0,0,1,0,0,0,0,0,0,...,1,1,1,0,0,1,0,0,NaN,0
1306,38.500000,0,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,1,NaN,1
1307,29.881138,0,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,1,NaN,1


In [154]:
corr_df = total_df_final.corr()
corr_df['Survived'].sort_values(ascending=False)

Survived        1.000000
Mrs             0.344935
Miss            0.332795
Pclass_1        0.285904
Family_mid      0.279855
Fare            0.257307
Cabin_B         0.175095
Embarked_C      0.168240
Cabin_D         0.150716
Cabin_E         0.145321
Cabin_C         0.114652
Pclass_2        0.093349
Master          0.085221
Cabin_F         0.057935
Cabin_A         0.022287
Family_size     0.016639
Cabin_G         0.016040
Royalty         0.016040
Embarked_Q      0.003650
Cabin_T        -0.026456
Officer        -0.031316
age            -0.070323
Embarked_S     -0.149683
Family_small   -0.203367
Cabin_U        -0.316912
Pclass_3       -0.322308
Sex            -0.543351
Mr             -0.549199
Family_large         NaN
Name: Survived, dtype: float64

In [155]:
# 找出其中相关性较强的特征主要是性别

In [250]:
# 选取其中相关性较强的的七个特征进行模型训练
X_total_df1 = pd.concat([Name_df_onehot,Family_df,df1[['Sex']],total_df[['Fare']],Pclass_df_df,Embarked_df_df,Cabin_One,total_df_final[['age']]],axis=1)
X_total_df1

,Master,Miss,Mr,Mrs,Officer,Royalty,Family_size,Family_small,Family_mid,Family_large,...,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,age
0,0,0,1,0,0,0,2,0,1,0,...,0,0,0,0,0,0,0,0,1,22.000000
1,0,0,0,1,0,0,2,0,1,0,...,0,0,1,0,0,0,0,0,0,38.000000
2,0,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,26.000000
3,0,0,0,1,0,0,2,0,1,0,...,0,0,1,0,0,0,0,0,0,35.000000
4,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,35.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,29.881138
1305,0,0,0,0,0,1,1,1,0,0,...,0,0,1,0,0,0,0,0,0,39.000000
1306,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,38.500000
1307,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,29.881138


### 划分测试集和训练集

训练集数据数据： (891, 12) 测试集数据数量： (418, 11)
总数据集 (1309, 12)

In [251]:
X_sorce = X_total_df1.loc[0:890,:]
X_sorce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 28 columns):
Master          891 non-null uint8
Miss            891 non-null uint8
Mr              891 non-null uint8
Mrs             891 non-null uint8
Officer         891 non-null uint8
Royalty         891 non-null uint8
Family_size     891 non-null int64
Family_small    891 non-null int64
Family_mid      891 non-null int64
Family_large    891 non-null int64
Sex             891 non-null int32
Fare            891 non-null float64
Pclass_1        891 non-null int32
Pclass_2        891 non-null int32
Pclass_3        891 non-null int32
Embarked_C      891 non-null int32
Embarked_Q      891 non-null int32
Embarked_S      891 non-null int32
Cabin_A         891 non-null uint8
Cabin_B         891 non-null uint8
Cabin_C         891 non-null uint8
Cabin_D         891 non-null uint8
Cabin_E         891 non-null uint8
Cabin_F         891 non-null uint8
Cabin_G         891 non-null uint8
Cabin_T         89

In [295]:
# 原始训练集
y_sorce = total_df_final.loc[0:890,'Survived']
# 原始预测集
pred_X = X_total_df1.loc[890:,:]

In [253]:
X_pred =X_total_df1.loc[891:,:]
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 891 to 1308
Data columns (total 28 columns):
Master          418 non-null uint8
Miss            418 non-null uint8
Mr              418 non-null uint8
Mrs             418 non-null uint8
Officer         418 non-null uint8
Royalty         418 non-null uint8
Family_size     418 non-null int64
Family_small    418 non-null int64
Family_mid      418 non-null int64
Family_large    418 non-null int64
Sex             418 non-null int32
Fare            418 non-null float64
Pclass_1        418 non-null int32
Pclass_2        418 non-null int32
Pclass_3        418 non-null int32
Embarked_C      418 non-null int32
Embarked_Q      418 non-null int32
Embarked_S      418 non-null int32
Cabin_A         418 non-null uint8
Cabin_B         418 non-null uint8
Cabin_C         418 non-null uint8
Cabin_D         418 non-null uint8
Cabin_E         418 non-null uint8
Cabin_F         418 non-null uint8
Cabin_G         418 non-null uint8
Cabin_T        

### 模型训练

In [265]:
# 通过sklearn自带方法发，划分测试集和训练集
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_sorce,y_sorce,test_size=0.2, random_state=42)

In [256]:
# 导入算法 ，通过逻辑回归建立模型，并通过fit方法训练模型
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [257]:
lin_reg.score(X_test,y_test)

0.4085205476269488

In [182]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())


In [258]:
# 线性 交叉验证
from sklearn.model_selection import cross_val_score
lin_reg1 = LinearRegression()
lin_scores = cross_val_score(lin_reg1, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [0.32604457 0.39349563 0.43248133 0.26722596 0.33765358 0.39048371
 0.41339999 0.3546093  0.40380045 0.28518712]
Mean: 0.36043816492478814
Standard deviation: 0.05287556917404348


In [268]:
# 使用决策树模型
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [273]:
tree_reg.score(X_test,y_test)

0.11148283852365481

In [288]:
# 随机森林模型
from sklearn.ensemble import RandomForestRegressor

forest_reg1 = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg1.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=42)

In [289]:
forest_reg1.score(X_test,y_test)

0.37670806574708937

## 实施方案

In [296]:
y_pred= lin_reg.predict(pred_X)

In [297]:
# 需转换成整型才可上传到kiggle网站
y_pred=y_pred.astype(int)

In [299]:
passenger_id = total_df_final.loc[890:,'PassengerId']

In [301]:
df_pred = pd.DataFrame( { 'PassengerId': passenger_id , 
     'Survived': y_pred } )

In [302]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 890 to 1308
Data columns (total 2 columns):
PassengerId    419 non-null int64
Survived       419 non-null int32
dtypes: int32(1), int64(1)
memory usage: 5.0 KB


In [303]:
df_pred.to_csv('titanic_pred.csv' , index = False )